In [131]:
#adaboost算法
import math
import numpy as np
import pandas as pd
x = [0,1,2,3,4,5,6,7,8,9]
y = [1,1,1,-1,-1,-1,1,1,1,-1]
w_1=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
data ={'x':x,'y':y,'w_1':w_1}
pd.DataFrame(data).T
def func_threshes(x):
    threshes = []
    for i in range(len(x)):
        threshes.append(x[i]-0.5)
    return threshes
threshes = func_threshes(x)

In [108]:
#计算预测值
def func_predict(x,y,threshe):
    pred_y_ps = []
    pred_y_ng = []
    for i in range(len(x)):
        if(x[i]<threshe):
            pred_y_ps.append(1)
            pred_y_ng.append(-1)
        else:
            pred_y_ps.append(-1)
            pred_y_ng.append(+1)
    return pred_y_ps,pred_y_ng
# pred_y_ps,pred_y_ng=func_predict(x,y,2.5)
# pred_y_ps,pred_y_ng

In [109]:
#定义计算错误率的函数，注意在使用的时候，需要判别从左往右，还是从右向左
def func_error(pred,y,w):
    error = 0
    error_num = 0
    right_num = 0
    w_e=[]
    for i in range(len(y)):
        if(pred[i]!=y[i]):
            error_num+=1
            w_e.append(w[i])
        else:
            right_num+=1
    error = round(error_num/(error_num+right_num),4)
    w_error = round(sum(w_e),4)#总误差
    return error,w_error


In [110]:
#选定阈值中是的误差最小的作为分类的值
def mini_thresh(w,x,y,threshes):
    good_thresh = 0
    eror_infor={}
    for thresh in threshes:
        pred_y_ps,pred_y_ng= func_predict(x,y,thresh)
        #判断从左向右，还是从右向左
        error_ps,w_error_ps=func_error(pred_y_ps,y,w)
        error_ng,w_error_ng=func_error(pred_y_ng,y,w)
        if w_error_ps>w_error_ng:
            w_error = w_error_ng
        else:
            w_error = w_error_ps            
        eror_infor[thresh]=round(w_error,4)  
    eror_infor1= min(eror_infor.items(),key=lambda x: x[1] )
    eror_index=eror_infor1[0]
    eror_value=eror_infor1[1]    
    return eror_infor,eror_index,eror_value
# eror_infor,eror_index,eror_value= mini_thresh(w_1,x,y,threshes)
# eror_index,eror_value

In [111]:
#选定好阈值后，需要计算分类器所取得权重
def func_alpha(w,x,y,threshes):
    alpha = 0
    #1调用函数选出需要的阈值
    eror_infor,eror_index,eror_value = mini_thresh(w,x,y,threshes)
    #2调用预测函数进行预测
    pred_y_ps,pred_y_ng = func_predict(x,y,eror_index)
    #3计算错误率
    error_ps,w_error_ps=func_error(pred_y_ps,y,w)
    error_ng,w_error_ng=func_error(pred_y_ng,y,w)
    if w_error_ps>w_error_ng:
        w_error = w_error_ng
    else:
        w_error = w_error_ps                   
    #4计算分类器的权重
    alpha_classification =round(0.5*math.log((1-w_error)/w_error),4)
    return alpha_classification
alpha_classification = func_alpha(w_1,x,y,threshes)
alpha_classification

0.4236

In [112]:
#更新数据训练的权值分布
def func_weight(w,x,y,threshes):
    #1计算出本轮的alpha的值
    alpha = func_alpha(w,x,y,threshes)
    #计算出本轮预测的值
    #2.1调用函数选出需要的阈值
    eror_infor,eror_index,eror_value = mini_thresh(w,x,y,threshes)
    #2.2调用预测函数进行预测
    pred_y_ps,pred_y_ng = func_predict(x,y,eror_index)
    
     #3计算错误率
    error_ps,w_error_ps=func_error(pred_y_ps,y,w)
    error_ng,w_error_ng=func_error(pred_y_ng,y,w)
    if w_error_ps>w_error_ng:
        w_error = w_error_ng
        pred_y = pred_y_ng

    else:
        w_error = w_error_ps   
        pred_y = pred_y_ps
    
    #2.3计算预测值与实际值与权重的乘积
    z_mi=[]
    for i in range(len(x)):
        z_mi.append(w[i]*math.exp(-alpha*y[i]*pred_y[i]))
    Z = sum(z_mi)
    #3.计算下一轮权重的情况
    w_mi=[]
    for i in range(len(x)):
        a = w[i]/Z*math.exp(-alpha*y[i]*pred_y[i])
        w_mi.append(round(a,5))
    W =w_mi
    return W
# w = func_weight(w_1,x,y,threshes)
# w_1= w
# w_1

[0.07143, 0.07143, 0.07143, 0.07143, 0.07143, 0.07143, 0.16666, 0.16666, 0.16666, 0.07143]

In [143]:
#定义adaboost函数，调用以上函数
def func_adaboost(w,x,y,threshes):
    alpha =[]
    for i in range(3):
        alpha.append(func_alpha(w,x,y,threshes))
        w = func_weight(w,x,y,threshes)
    return alpha,w


In [144]:
alpha,w=func_adaboost(w_1,x,y,threshes)
alpha,w

([0.4236, 0.6496, 0.7521], [0.12502, 0.12502, 0.12502, 0.10184, 0.10184, 0.10184, 0.06481, 0.06481, 0.06481, 0.12502])